In [1]:
import pandapower.networks as pn
import os
import pandapower as pp
import polars as pl
from polars import col as c
from shapely import  from_geojson
from plotly.graph_objects import Figure
import plotly.graph_objs as go
from general_function import pl_to_dict, build_non_existing_dirs

from data_display.grid_plotting import plot_grid_from_pandapower
from data_connector import pandapower_to_dig_a_plan_schema
from polars_function import list_to_list_of_tuple
from polars_function import keep_only_duplicated_list, keep_only_first_unique_list

os.chdir(os.getcwd().replace("/src", ""))


In [2]:
net = pn.mv_oberrhein()
net["ext_grid"] = net["ext_grid"].iloc[0:1]

In [3]:

bus: pl.DataFrame = pl.from_pandas(net["bus"], include_index=True).rename({"None": "bus_id"})


line: pl.DataFrame = pl.from_pandas(net["line"], include_index=True).rename({"None": "line_id"})


line = line.filter(pl.concat_list("from_bus", "to_bus").pipe(keep_only_first_unique_list))\
    .drop("name").with_row_index(name="name")\
    .with_columns(
        ("Branche_" + c("name").cast(pl.Utf8)).alias("name")
    ).join(
        bus.select("bus_id", "vn_kv"),
        left_on="from_bus",
        right_on="bus_id",
        how="left"
)
trafo: pl.DataFrame = pl.from_pandas(net["trafo"]).drop("name").with_row_index(name="name").with_columns(
    ("Trafo_" + c("name").cast(pl.Utf8)).alias("name")
)
max_bus_id = bus["bus_id"].max()
switch: pl.DataFrame = pl.from_pandas(net["switch"]).with_row_index(offset=max_bus_id + 1, name="new_bus_id")

new_bus_mapping = pl_to_dict(switch.select(
    pl.concat_list("element", "bus").cast(pl.List(pl.Utf8)).list.join("_"), 
    "new_bus_id"
))

line = line.with_columns(
    pl.concat_list("line_id", col).cast(pl.List(pl.Utf8)).list.join("_").replace_strict(new_bus_mapping, default=c(col)).alias(col)
    for col in ["from_bus", "to_bus"]
)

switch = switch.with_columns(
    c("new_bus_id").alias("element"),
    pl.lit("b").alias("et")
)

In [4]:
bus = pl.concat([
    bus,
    switch.select(
    c("element").alias("bus_id"),
    ("Bus " + c("element").cast(pl.Utf8)).alias("name"),
    pl.lit(20).alias("vn_kv"),
    pl.lit("b").alias("type"),
    pl.lit("1.0").alias("zone"),
    pl.lit(True).alias("in_service"),
)
], how="diagonal_relaxed")


net["bus"] = bus.to_pandas().set_index("bus_id", drop=True)
net["switch"] = switch.drop(["new_bus_id"]).to_pandas()
net["line"] = line.drop("vn_kv").to_pandas()
net["trafo"] = trafo.to_pandas()
net["ext_grid"]["vm_pu"] = 1.0
net["trafo"]["shift_degree"] = 0


build_non_existing_dirs(".cache/input_data")
pp.set_isolated_areas_out_of_service(net, respect_switches=False)
pp.drop_out_of_service_elements(net)
pp.to_pickle(net, ".cache/input_data/mv_example_oberrhein.p")

2025-06-23 09:41:12 B01BPC15 pandapower.toolbox.grid_modification[29432] INFO set 0 of 0 unsupplied buses out of service
